In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
img_size = 48
batch_size = 32
epochs = 20
learning_rate = 3e-4
step_size = 5
gamma = 0.1
data_dir = '/content/drive/MyDrive/dataset'
save_path = '/content/drive/MyDrive/race.pth'

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
rgb_mean = [0.485, 0.456, 0.406]
rgb_std = [0.229, 0.224, 0.225]

Computed RGB mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [ ]:
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(img_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])
test_tf = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(rgb_mean, rgb_std),
])

In [ ]:
# data
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

classes = train_dataset.classes
num_classes = len(classes)

In [ ]:
# model
import timm
model = timm.create_model(
    'mobilevit_s',
    pretrained=True,
    num_classes=num_classes,
    in_chans=3
).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

In [ ]:
# Loss, Optimizer, Scheduler
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# train
best_acc = 0.0
for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    scheduler.step()
    epoch_loss = running_loss / len(train_dataset)

    model.eval()
    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    test_acc = correct / total * 100
    print(f"Epoch {epoch:02d}: Train Loss={epoch_loss:.4f}, Test Acc={test_acc:.2f}%")
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), save_path)
        print(f" New best model saved with acc={best_acc:.2f}%")
print(f"Best Test Accuracy: {best_acc:.2f}%")

Epoch 01: Train Loss=1.1772, Test Acc=62.99%
 New best model saved with acc=62.99%
Epoch 02: Train Loss=0.9791, Test Acc=64.66%
 New best model saved with acc=64.66%
Epoch 03: Train Loss=0.9235, Test Acc=66.48%
 New best model saved with acc=66.48%
Epoch 04: Train Loss=0.8818, Test Acc=66.82%
 New best model saved with acc=66.82%
Epoch 05: Train Loss=0.8557, Test Acc=69.35%
 New best model saved with acc=69.35%
Epoch 06: Train Loss=0.7980, Test Acc=69.82%
 New best model saved with acc=69.82%
Epoch 07: Train Loss=0.7855, Test Acc=70.48%
 New best model saved with acc=70.48%
Epoch 08: Train Loss=0.7736, Test Acc=70.38%
Epoch 09: Train Loss=0.7765, Test Acc=70.85%
 New best model saved with acc=70.85%
Epoch 10: Train Loss=0.7603, Test Acc=70.43%
Epoch 11: Train Loss=0.7582, Test Acc=70.51%
Epoch 12: Train Loss=0.7536, Test Acc=70.65%
Epoch 13: Train Loss=0.7477, Test Acc=70.97%
 New best model saved with acc=70.97%
Epoch 14: Train Loss=0.7504, Test Acc=71.05%
 New best model saved with a

In [ ]:
# predict
from PIL import Image
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

def predict(img_path):
    img = Image.open(img_path).convert('RGB')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
    idx = logits.argmax(dim=1).item()
    return classes[idx]

# pred = predict('/content/drive/MyDrive/lgh.jpg')
# print('Predicted race:', pred)

Predicted race: White
